<img src="../assets/logos/Logo_layout.png" width=500></img>


# Análisis
Tu primer modelo simplemente coloca barcos de tamaño 2 aleatoriamente. Dado que puedes fabricar barcos de muchos tamaños diferentes, nos gustaría saber cuál es la mejor forma. Quizá también deberás cambiar la forma de tus barcos, ya que los modelos de disparo de otros equipos aprenderán los patrones de tu esquema. En este modelo, en cada juego, tu modelo elegirá una de dos estrategias de distribución diferentes y luego usarás [Amazon Athena](https://aws.amazon.com/athena/) para ver los resultados. Puedes repetir el experimento con nuevos tamaños y formas de barcos para ver si puedes encontrar mejores estrategias.

## Código 
El código de tu segundo modelo está en /containers/Layout/2vs3 y contiene los siguientes archivos
    
- ### /containers/Layout/2vs3/Makefile
Makefile para crear el archivo tar.gz del código para los contenedores del entorno de trabajo de sagemaker

- ### [/containers/Layout/2vs3/host.py](/edit/GameDayRepo/containers/Layout/2vs3/host.py)
El código para generar inferencias. Además de seleccionar aleatoriamente una estrategia de distribución, también colocará una etiqueta en el atributo de la sesión que usaremos en athena para comparar diferentes juegos. Puedes cambiar el valor de la etiqueta (lo cual deberías hacer por cada modelo) editando el archivo host.py. (no ejecutes este archivo directamente, usa test.py)

- ### [/containers/Layout/2vs3/test.py](/edit/GameDayRepo/containers/Layout/2vs3/test.py)
código para probar los archivos host.py y train.py localmente. Crea una simulación de tiempo de ejecución de sagemaker para ejecutar train.py y luego test.py. Puedes editar este archivo para personalizar tu prueba. Si recibes un mensaje acerca de dependencias faltantes (como mxnet) simplemente instálalas con:
```shell
pip install mxnet # for example
```

- ### [/containers/Layout/2vs3/train.py](/edit/GameDayRepo/containers/Layout/2vs3/train.py)
el código de entrenamiento (no ejecutes este archivo directamente, usa test.py)


A continuación, ejecuta el script de prueba localmente para confirmar que el script funciona.
```shell
python ./test.py
```

## Configurar
Antes de poder implementar tu segundo modelo, debes configurar la implementación. Como se describe en el [Cuadernillo de punto final](../Endpoint_Reference.ipynb), editarás para la configuración de la implementación en [/layout-config.json](/edit/GameDayRepo/layout-config.json). deberás editar el siguiente campo según los siguientes valores

|campo de parámetro|Valor|
|---|---|
|trainsourcefile|`layout-2vs3.tar.gz`|
|hostsourcefile|`layout-2vs3.tar.gz`|



## Implementar
1. sigue las instrucciones desde la referencia del punto final para aplicar tu código y los cambios, y luego transferir tus cambios al módulo remoto
2. indica a operaciones que implementen tus cambios

## Siguientes pasos
A continuación, monitorea el desempeño de tu modelo. Consulta [Athena](../Athena.ipynb) para obtener más detalles sobre cómo configurar Athena (reemplaza los valores entre <> como en las instrucciones de Athena). La siguiente consulta mostrará el porcentaje ganado por tamaño de barco (2 o 3).
```sql
SELECT type,
         sum(won)/(sum(won)+sum(lost)) as won,
         sum(lost)/(sum(won)+sum(lost)) as lost
FROM 
    (SELECT json_extract(session,'$.size') AS size,
        CASE
            WHEN winner = '<Team-Name>' THEN 1.0
            ELSE 0.0
        END AS won,
        CASE
            WHEN winner != '<Team-Name>' THEN 1.0
            ELSE 0.0
        END AS lost
    FROM 
        (SELECT teama.session AS session,
         teama.teamname AS team,
         winner
        FROM "<Game-data-catalog>"."<Game-data-bucket>"
        UNION
        SELECT teamb.session AS session,
         teamb.teamname AS team,
         winner
        FROM "<Game-data-catalog>"."<Game-data-bucket>" )
        WHERE team = '<Team-Name>' )
    GROUP BY  size
``` 
Ahora programa algunos nuevos tamaños o formas de barcos para hacer otra ronda de experimentos, impleméntalos, y vuelve a ejecutar tu consulta de athena para ver el desempeño